In [1]:
import pandas as pd
import altair as alt


alt.data_transformers.disable_max_rows()

url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/ufo-scrubbed-geocoded-time-standardized-00.csv"

columns = [
    "datetime", "city", "state", "country", "shape", "duration_seconds",
    "duration_reported", "summary", "date_posted", "latitude", "longitude"
]

ufo = pd.read_csv(url, names=columns, header=0)  

ufo['datetime'] = pd.to_datetime(ufo['datetime'], errors='coerce')

ufo['year'] = ufo['datetime'].dt.year
ufo = ufo[ufo['datetime'].dt.year >= 1980]
ufo = ufo.dropna(subset=['year'])

ufo.head()

,datetime,city,state,country,shape,duration_seconds,duration_reported,summary,date_posted,latitude,longitude,year
11,1970-10-10 16:00:00,bellmore,ny,us,disk,1800.0,30 min.,silver disc seen by family and neighbors,5/11/2000,40.668611,-73.527500,1970
12,1970-10-10 19:00:00,manchester,ky,us,unknown,180.0,3 minutes,Slow moving&#44 silent craft accelerated at an...,2/14/2008,37.153611,-83.761944,1970
13,1971-10-10 21:00:00,lexington,nc,us,oval,30.0,30 seconds,green oval shaped light over my local church&#...,2/14/2010,35.823889,-80.253611,1971
14,1972-10-10 19:00:00,harlan county,ky,us,circle,1200.0,20minutes,On october 10&#44 1972 myself&#44my 5yrs.daugh...,9/15/2005,36.843056,-83.321944,1972
15,1972-10-10 22:30:00,west bloomfield,mi,us,disk,120.0,2 minutes,The UFO was so close&#44 my battery in the car...,8/14/2007,42.537778,-83.233056,1972


In [2]:
ufo['datetime'] = pd.to_datetime(ufo['datetime'], errors='coerce')
ufo['year'] = ufo['datetime'].dt.year
year_counts = ufo['year'].value_counts().sort_index().reset_index()
year_counts.columns = ['year', 'count']

ufochart1 = alt.Chart(year_counts).mark_line().encode(
    x='year:O',
    y='count:Q'
).properties(
    title='UFO Sightings per Year (after 1970)',
    width=500
)


In [3]:
ufo_filtered = ufo.dropna(subset=['shape', 'state'])


state_dropdown = alt.binding_select(options=sorted(ufo_filtered['state'].unique()))
state_select = alt.selection_single(fields=['state'], bind=state_dropdown, name="Select")

ufochart2 = alt.Chart(ufo_filtered).mark_bar().encode(
    x=alt.X('shape:N', title='UFO Shape'),
    y=alt.Y('count():Q', title='Number of Sightings'),
    color=alt.Color('shape:N', legend=None),
    tooltip=['shape', 'count()']
).transform_filter(
    state_select
).properties(
    width=600,
    height=400,
    title='UFO Shapes by State (Interactive)'
).add_params(
    state_select
)



/var/folders/4y/gfvp3jps5710ysvdz003nhnc0000gn/T/ipykernel_34311/1885883382.py:5: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  state_select = alt.selection_single(fields=['state'], bind=state_dropdown, name="Select")


In [4]:

ufochart1.save('ufochart1.json')

ufochart2.save('ufochart2.json')
